In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import pymysql
import calendar
import time
import json
import requests
from datetime import datetime

In [2]:
class DBUpdater:
    def __init__(self):
        self.conn = pymysql.connect(
            host='localhost', user='root', password='password', db='analyzer', charset='utf8')

        with self.conn.cursor() as curs:
            sql = """
            CREATE TABLE IF NOT EXISTS company_info (
                code VARCHAR(20),
                company VARCHAR(40),
                sector VARCHAR(50),
                last_update DATE,
                PRIMARY KEY (code)) ENGINE=InnoDB DEFAULT CHARSET=utf8;
            """
            curs.execute(sql)
            sql = """
            CREATE TABLE IF NOT EXISTS daily_price (
                code VARCHAR(20),
                date DATE,
                open BIGINT(20),
                high BIGINT(20),
                low BIGINT(20),
                close BIGINT(20),
                diff BIGINT(20),
                volume BIGINT(20),
                PRIMARY KEY (code, date)) ENGINE=InnoDB DEFAULT CHARSET=utf8;
            """
            curs.execute(sql)
        self.conn.commit()
        self.codes = dict()

    def __del__(self):
        self.conn.close()
        
    def read_krx(self):
        url = 'http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13'
        krx = pd.read_html(url, header=0)[0]
        krx = krx[['종목코드', '회사명', '업종']]
        krx = krx.rename(columns={'종목코드': 'code', '회사명': 'company', '업종': 'sector'})
        krx.code = krx.code.map('{:06d}'.format)
        return krx

    def update_company_info(self):
        sql = "SELECT * FROM company_info"
        df = pd.read_sql(sql, self.conn)
        for idx in range(len(df)):
            self.codes[df['code'].values[idx]] = df['company'].values[idx]

        with self.conn.cursor() as curs:
            sql = "SELECT max(last_update) FROM company_info"
            curs.execute(sql)
            rs = curs.fetchone()
            today = datetime.today().strftime('%Y-%m-%d')
            if rs[0] == None or rs[0].strftime('%Y-%m-%d') < today:
                krx = self.read_krx()
                for idx in range(len(krx)):
                    code = krx.code.values[idx]
                    company = krx.company.values[idx]
                    sector = krx.sector.values[idx]
                    sql = f"REPLACE INTO company_info (code, company, sector, last_update) VALUES ('{code}', '{company}', '{sector}', '{today}')"
                    curs.execute(sql)
                    self.codes[code] = company
                    date_time = datetime.now().strftime('%Y-%m-%d %H:%M')
                    print(
                        f"[{date_time}] #{idx+1:04d} REPLACE INTO company_info VALUES ({code}, {company}, {sector}, {today})")
                self.conn.commit()
                print('*')
        
    def execute_daily(self):
        self.update_company_info()

In [3]:
dbu = DBUpdater()
dbu.update_company_info()

In [4]:
from urllib.request import urlopen

url = f"http://finance.naver.com/item/sise_day.nhn?code=000660"
html = BeautifulSoup(requests.get(url, headers={'User-agent': 'Mozilla/5.0'}).text, "lxml")
pgrr = html.find("td", class_="pgRR")

In [5]:
print(pgrr.prettify())

<td class="pgRR">
 <a href="/item/sise_day.nhn?code=000660&amp;page=613">
  맨뒤
  <img alt="" border="0" height="5" src="https://ssl.pstatic.net/static/n/cmn/bu_pgarRR.gif" width="8"/>
 </a>
</td>



In [6]:
s = str(pgrr.a["href"]).split('=')
print(s)

['/item/sise_day.nhn?code', '000660&page', '613']


In [7]:
last_page = s[-1]
last_page

'613'

In [8]:
df = pd.DataFrame()
sise_url = "http://finance.naver.com/item/sise_day.nhn?code=000660"

for page in range(1, int(last_page) +1):
    page_url = '{}&page={}'.format(sise_url, page)
    df = df.append(pd.read_html(requests.get(page_url,
                    headers={'User-agent': 'Mozilla/5.0'}).text)[0])    
    
df = df.dropna()
print(df)

            날짜        종가      전일비        시가        고가        저가        거래량
1   2021.06.11  128000.0   5000.0  124500.0  128500.0  124000.0  4535613.0
2   2021.06.10  123000.0    500.0  122500.0  124000.0  120500.0  4533076.0
3   2021.06.09  122500.0   5000.0  125500.0  126000.0  122500.0  6087592.0
4   2021.06.08  127500.0   1000.0  127000.0  127500.0  125000.0  4194955.0
5   2021.06.07  128500.0      0.0  129500.0  130000.0  128000.0  2562918.0
..         ...       ...      ...       ...       ...       ...        ...
11  1997.01.06   20000.0      0.0   20000.0   20000.0   20000.0   436190.0
12  1997.01.04   20000.0      0.0   20000.0   20000.0   20000.0   277690.0
13  1997.01.03   20000.0   3000.0   20000.0   20100.0   20000.0   145680.0
1   1996.12.27   23000.0      0.0   23000.0   23000.0   23000.0        0.0
2   1996.12.26   23000.0  23000.0   23000.0   23000.0   23000.0        0.0

[6122 rows x 7 columns]


In [9]:
from urllib.request import urlopen

url = f"https://finance.naver.com/item/frgn.nhn?code=000660"
html = BeautifulSoup(requests.get(url, headers={'User-agent': 'Mozilla/5.0'}).text, "lxml")
pgrr = html.find("td", class_="pgRR")

In [10]:
print(pgrr.prettify())

<td class="pgRR">
 <a href="/item/frgn.nhn?code=000660&amp;page=204">
  맨뒤
  <img alt="" border="0" height="5" src="https://ssl.pstatic.net/static/n/cmn/bu_pgarRR.gif" width="8"/>
 </a>
</td>



In [11]:
s = str(pgrr.a["href"]).split('=')
print(s)

['/item/frgn.nhn?code', '000660&page', '204']


In [15]:
last_page = s[-1]
last_page

'204'

In [17]:
df_V = pd.DataFrame()
sise_url = "https://finance.naver.com/item/frgn.nhn?code=000660"

for page in range(1, int(last_page) +1):
    page_url = '{}&page={}'.format(sise_url, page)
    df_V = df_V.append(pd.read_html(requests.get(page_url,
                    headers={'User-agent': 'Mozilla/5.0'}).text)[0])    
    
df_V = df_V.dropna()

In [13]:
df_V

,0,1,2
0,"전일 123,000 123,000","고가 128,500128,500 (상한가 159,500159,500 )","거래량 4,535,613 4,535,613"
1,"시가 124,500124,500","저가 124,000124,000 (하한가 86,500 )","거래대금 574,198 574,198 백만"
0,"전일 123,000 123,000","고가 128,500128,500 (상한가 159,500159,500 )","거래량 4,535,613 4,535,613"
1,"시가 124,500124,500","저가 124,000124,000 (하한가 86,500 )","거래대금 574,198 574,198 백만"
0,"전일 123,000 123,000","고가 128,500128,500 (상한가 159,500159,500 )","거래량 4,535,613 4,535,613"
...,...,...,...
1,"시가 124,500124,500","저가 124,000124,000 (하한가 86,500 )","거래대금 574,198 574,198 백만"
0,"전일 123,000 123,000","고가 128,500128,500 (상한가 159,500159,500 )","거래량 4,535,613 4,535,613"
1,"시가 124,500124,500","저가 124,000124,000 (하한가 86,500 )","거래대금 574,198 574,198 백만"
0,"전일 123,000 123,000","고가 128,500128,500 (상한가 159,500159,500 )","거래량 4,535,613 4,535,613"
